In [ ]:
from compounds_research.compound.utils import plot_utilization_vs_rate, plot_raw_data,  plot_market_util, plot_market_rates, c_markets, make_rates_df, get_comp_market
from compounds_research.compound.fetch_compound import make_dataframe, token_addresses, variables
from compounds_research.utils import get_market

from compounds_research.aave import analyze_aave
from compounds_research.dydx import analyze_dydx

from compounds_research import settings
from matplotlib.lines import Line2D

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
plt.style.use('ggplot')
plt.rc('axes',edgecolor='darkgrey')
plt.rcParams["axes.linewidth"]  = 0.5

In [ ]:
markets = {
    'dydx': ['dai', 'eth', 'usdc'],
    'compound': ['dai', 'eth', 'bat', 'zrx', 'usdc', 'sai', 'rep', 'wbtc'],
    'aave': ['LEND', 'LINK', 'DAI', 'USDC', 'WBTC', 'sUSD', 'KNC', 'USDT',
       'ETH', 'SNX', 'TUSD', 'MANA', 'BAT', 'MKR', 'BUSD', 'ZRX', 'REP']
}

all_markets = ['DAI', 'ETH', 'USDC', 'WBTC', 'BAT', 'ZRX', 'REP', 'SAI', 
               'LEND', 'LINK', 'USDT', 'KNC', 'SNX', 'TUSD', 'MANA', 'MKR', 'BUSD']

In [ ]:
# prices: coinmarketcap, access: 22-05-2020
usd_markets = {
    'dai': 1,
    'eth': 203.01,
    'bat': 0.20329,
    'zrx': 0.33876,
    'sai': 1.09,
    'usdc': 1,
    'rep': 12.38,
    'wbtc': 9197.24,
    'lend': 0.057779,
    'link': 3.96,
    'knc': 0.634943,
    'usdt': 1,
    'mana': 0.036948,
    'mkr': 316.95,
    'busd': 1
}

## Data Exploration
This notebook examines a given market across the different lending platforms Aave, Compound and dYdX.

### Load Data

In [ ]:
aave = analyze_aave.load_data()
aave

In [ ]:
def get_median_usd_funds_locked(platform: str, market: str):
    if platform == 'aave':
        return aave[(aave['Symbol'] == market.upper()) & (aave['Datetime'] <= '2020-05-07 00:00:00')]['Total Liquidity'].median() * usd_markets[market.lower()]
    elif platform == 'compound':
        df_comp = get_market(market.lower(), 'compound')
        return df_comp[df_comp.index <= '2020-05-07 00:00:00']['total_supply_history'].median() * usd_markets[market.lower()]
    elif platform == 'dydx':
        df_dydx = analyze_dydx.load_dydx_market(market.lower())
        return df_dydx[df_dydx.index <= '2020-05-07 00:00:00']['supply_amount'].median() * usd_markets[market.lower()]

In [ ]:
plt.plot(aave[aave['Symbol'] == 'DAI']['Datetime'], aave[aave['Symbol'] == 'DAI']['Utilization Rate'])

In [ ]:
comp = get_market('eth', 'compound')
comp

In [ ]:
df_eth = analyze_dydx.load_dydx_market('eth')

### Methods (plotting)

In [ ]:
def plot_borrow_rates(market: str):
    lwidth = 0.8
    fsize = 10
    plt.figure(figsize=(12, 8))
    plt.ylabel('Interest Rate', fontdict={'size': fsize})
    if market in markets['compound']:
        df_comp = get_market(market.lower(), 'compound')
        plt.plot(df_comp['borrow_rates'], label='Compound', linewidth=lwidth, color='steelblue')
    if market.upper() in markets['aave']:
        plt.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Variable Borrow Rate'], label='Aave', linewidth=lwidth, color='tomato')
    if market in markets['dydx']:
        df_dydx = analyze_dydx.load_dydx_market(market.lower())
        plt.plot(df_dydx['interest_rate'], label='dYdX', linewidth=lwidth, color='lightseagreen')
    ax = plt.gca()
    ax.set_facecolor('white')
    ax.grid(color='whitesmoke')
    plt.title(market.capitalize()+': Borrow Interest Rates', fontdict={'size':fsize})
    plt.legend(fontsize=fsize)
    plt.show()
    

In [ ]:
def plot_liquidity(market: str, plot_aave=True, plot_comp=True, plot_dydx=True):
    lwidth = 1
    fsize = 10
    plt.figure(figsize=(12, 8))
    plt.ylabel(market.capitalize(), fontdict={'size': fsize})
    if plot_comp:
        df_comp = get_market(market.lower(), 'compound')
        plt.plot(df_comp['total_borrows_history'], label='Borrow', linewidth=lwidth, color='tomato')
        plt.plot(df_comp['total_supply_history'], label='Supply', linewidth=lwidth, color='steelblue')
        plt.fill_between(df_comp.index, df_comp['total_borrows_history'], df_comp['total_supply_history'], color='aliceblue')
        plt.legend()
    if plot_aave:
        plt.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Total Borrows'],  label='Borrow', linewidth=lwidth, color='tomato')
        plt.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Total Liquidity'], label='Supply', linewidth=lwidth, color='steelblue')
        plt.fill_between(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Total Borrows'],aave[aave['Symbol'] == market.upper()]['Total Liquidity'], color='mistyrose')
    if plot_dydx:
        df_dydx = analyze_dydx.load_dydx_market(market.lower())
        plt.plot(df_dydx['borrow_amount'], label='Borrow', linewidth=lwidth, color='tomato')
        plt.plot(df_dydx['supply_amount'], label='Supply', linewidth=lwidth, color='steelblue')
        plt.fill_between(df_dydx.index, df_dydx['borrow_amount'], df_dydx['supply_amount'], color='peachpuff')
    plt.title('Liquidity', fontdict={'size':fsize})
    plt.yscale('log')
    
    
    comp_patch = mpatches.Patch(color='aliceblue', label='Compound')
    aave_patch = mpatches.Patch(color='mistyrose', label='Aave')
    dydx_patch = mpatches.Patch(color='peachpuff', label='dYdX')
    supply_line = Line2D([0], [0], color='steelblue', lw=2, label='Supply')
    borrows_line = Line2D([0], [0], color='tomato', lw=2, label='Borrows')
    plt.legend(handles=[supply_line, borrows_line, comp_patch, aave_patch, dydx_patch], prop={'size':fsize}, handlelength=3)
    
    ax = plt.gca()
    ax.set_facecolor('white')
    ax.grid(color='whitesmoke')
    plt.show()


In [ ]:
def plot_liquidity_util(market: str, plot_aave=False, plot_comp=False, plot_dydx=False):
    lwidth = 0.8
    fsize = 10
    #plt.figure(figsize=(12, 8))
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()
    ax1.set_ylabel(market.capitalize(), fontdict={'size': fsize})
    ax2.set_ylabel('Utilization', fontdict={'size': fsize})
    
    if plot_comp:
        df_comp = get_market(market.lower(), 'compound')
        ax1.plot(df_comp['total_borrows_history'], label='Borrow', linewidth=lwidth, color='tomato')
        ax1.plot(df_comp['total_supply_history'], label='Supply', linewidth=lwidth, color='steelblue')
        ax2.plot(df_comp['utilization_ratio'], label='Utilization', linewidth=lwidth, linestyle='--', color='dodgerblue')
       
    elif plot_aave:
        ax1.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Total Borrows'],  label='Borrow', linewidth=lwidth, color='tomato')
        ax1.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Total Liquidity'], label='Supply', linewidth=lwidth, color='steelblue')
        ax2.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Utilization Rate'], label='Utilization', linewidth=lwidth, linestyle='--', color='dodgerblue')
    elif plot_dydx:
        df_dydx = analyze_dydx.load_dydx_market(market.lower())
        ax1.plot(df_dydx['borrow_amount'], label='Borrow', linewidth=lwidth, color='tomato')
        ax1.plot(df_dydx['supply_amount'], label='Supply', linewidth=lwidth, color='steelblue')
        ax2.plot(df_dydx['utilization'], label=market, linewidth=lwidth, linestyle='--', color='dodgerblue')

    
#     comp_patch = mpatches.Patch(color='aliceblue', label='Compound')
#     aave_patch = mpatches.Patch(color='mistyrose', label='Aave')
#     dydx_patch = mpatches.Patch(color='peachpuff', label='dYdX')
#     supply_line = Line2D([0], [0], color='steelblue', lw=2, label='Supply')
#     borrows_line = Line2D([0], [0], color='tomato', lw=2, label='Borrows')
#     plt.legend(handles=[supply_line, borrows_line, comp_patch, aave_patch, dydx_patch], prop={'size':fsize}, handlelength=3)
    
    ax1.set_facecolor('white')
    ax1.grid(color='whitesmoke')
    plt.show()


In [ ]:
plot_liquidity_util('dai', plot_comp=True)
plot_liquidity_util('dai', plot_aave=True)
plot_liquidity_util('dai', plot_dydx=True)

In [ ]:
def plot_utilization(market: str, plot_aave=True, plot_comp=True, plot_dydx=True):
    lwidth = 0.8
    fsize = 10
    plt.figure(figsize=(12, 8))
    plt.ylabel('Utilization', fontdict={'size': fsize})
    if plot_comp and market in markets['compound']:
        df_comp = get_market(market.lower(), 'compound')
        plt.plot(df_comp['utilization_ratio'], label='Compound', linewidth=lwidth, color='steelblue')
    if plot_aave and market.upper() in markets['aave']:
        plt.plot(aave[aave['Symbol'] == market.upper()]['Datetime'], aave[aave['Symbol'] == market.upper()]['Utilization Rate'], label='Aave', linewidth=lwidth, color='slategrey')
    if plot_dydx and market in markets['dydx']:
        df_dydx = analyze_dydx.load_dydx_market(market.lower())
        plt.plot(df_dydx['utilization'], label='dYdX', linewidth=lwidth, color='lightcoral')
    plt.legend(fontsize=fsize)
    ax = plt.gca()
    ax.set_facecolor('white')
    ax.grid(color='whitesmoke')
    plt.title(market.capitalize(), fontdict={'size': fsize})
    plt.show()
    
def plot_utilization_per_platform(platform: str):
    lwidth = 1
    fsize = 10
    plt.figure(figsize=(12, 8))
    for market in markets[platform]:
        if platform == 'compound':
            df_comp = get_market(market.lower(), 'compound')
            plt.plot(df_comp['utilization_ratio'], label=market, linewidth=lwidth)
        elif platform == 'aave':
            plt.plot(aave[aave['Currency'] == market.upper()]['Datetime'], aave[aave['Currency'] == market.upper()]['Utilization Rate'], label=market, linewidth=lwidth)
        elif platform == 'dydx':
            df_dydx = analyze_dydx.load_dydx_market(market.lower())
            plt.plot(df_dydx['utilization'], label=market, linewidth=lwidth)
        else:
            print('invalid platform: ', platform)
            return
    plt.ylabel('Utilization', fontdict={'size': fsize})
    plt.legend(fontsize=fsize)
    ax = plt.gca()
    ax.set_facecolor('white')
    ax.grid(color='whitesmoke')
    plt.title(platform.capitalize(), fontdict={'size': fsize})
    plt.show()
    

In [ ]:
msize=2.5
fsize=6

# plot: all markets per platform: show median locked funds, avg. utilization, avg. borrow rate
def plot_market_summaries():
    fig = plt.figure(figsize=(1*3.57072,1*2.75))
    gs = matplotlib.gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[1, 1], hspace=0.22)
    ax1 = fig.add_subplot(gs[1,0])
    ax2 = fig.add_subplot(gs[0,0], sharex=ax1)

    # plot for aave
    for market in all_markets:
        if market.upper() not in markets['aave']:
            continue
        ax1.plot(market, aave[aave['Symbol'] == market.upper()]['Utilization Rate'].mean(), 'r', marker='x', markersize=msize)
        ax2.plot(market, aave[aave['Symbol'] == market.upper()]['Variable Borrow Rate'].mean(), 'ro', markersize=msize)
        
    # plot for dydx
    for market in all_markets:
        if market.lower() not in markets['dydx']:
            continue
        dydx_df = analyze_dydx.load_dydx_market(market.lower())
        ax1.plot(market, dydx_df['utilization'].mean(), 'o', marker='x', color='dodgerblue', markersize=msize)
        ax2.plot(market,dydx_df['interest_rate'].mean(), 'o', color='dodgerblue', markersize=msize)
        
    # plot for compound
    for market in all_markets:
        if market.lower() not in markets['compound']:
            continue
        comp_df = get_market(market.lower(), 'compound')
        ax1.plot(market, comp_df['utilization_ratio'].mean(), 'o', marker='x', color='slategrey', markersize=msize)
        ax2.plot(market,comp_df['borrow_rates'].mean(), 'o', color='slategrey', markersize=msize)
        
    ax1.set_ylabel('Avg.\n Utilization', fontdict={'size':fsize})
    ax1.set_ylim(0,1)
    plt.setp(ax2.get_xticklabels(), visible=False)
    #plt.setp(ax1.get_xticklabels(), visible=False)
    ax1.tick_params(axis='x', labelrotation=90)
    ax2.set_ylabel('Avg.\n Borrow Rate', fontdict={'size':fsize})
    ax2.legend()
    
    custom_lines = [Line2D([0], [0], color='r', lw=3),
                Line2D([0], [0], color='dodgerblue', lw=3),
                Line2D([0], [0], color='slategrey', lw=3)]

    ax2.legend(custom_lines, ['Aave', 'dYdX', 'Compound'], fontsize=fsize)
    ax2.set_ylim(0,0.3)
    ax1.set_facecolor('white')
    ax1.grid(color='whitesmoke')
    ax2.set_facecolor('white')
    ax2.grid(color='whitesmoke')
    ax1.yaxis.get_offset_text().set_fontsize(4)
    ax1.tick_params(axis='both', which='major', labelsize=fsize, width=0.25)
    ax1.tick_params(axis='both', which='minor', labelsize=fsize)
    ax2.tick_params(axis='both', which='major', labelsize=fsize, width=0.25)
    ax2.tick_params(axis='both', which='minor', labelsize=fsize)
    plt.savefig('platforms.pdf', dpi=900)

In [ ]:
plot_market_summaries()

### Interest rates and utilization

In [ ]:
# intersecting markets for Compound, Aave and dYdX
for market in list(set(markets['compound']) & set(markets['dydx']) & set(map(lambda m: m.lower(), markets['aave']))):
    plot_borrow_rates(market)

### Liquidity and illiquidity

In [ ]:
plot_liquidity('dai')

In [ ]:
plot_liquidity('eth')

In [ ]:
plot_liquidity('usdc')

### Platform utilization

In [ ]:
plot_utilization_per_platform('compound')
plot_utilization_per_platform('aave')
plot_utilization_per_platform('dydx')

### Cross-platform utilization 

In [ ]:
# intersecting markets for Compound, Aave and dYdX
for market in list(set(markets['compound']) & set(markets['dydx']) & set(map(lambda m: m.lower(), markets['aave']))):
    plot_utilization(market)